In [1]:
import twint
import pandas as pd
import nest_asyncio
nest_asyncio.apply()
pd.options.display.max_columns = 40


In [2]:
import datetime as dt
import time
import random

def arrange_dates(start_date, end_date):
    """Used in the get_election_tweets func. """
    
    dates = []
    delta = end_date - start_date
    
    for i in range(delta.days + 1):
        day = start_date + dt.timedelta(days=i)
        dates.append(day)
    
    return dates

def get_election_tweets(tweets_per_day=5000, start_date="2020-10-01", end_date="2020-11-03"):
    
    # get a list of dates
    f = "%Y-%m-%d"
    sd = dt.datetime.strptime(start_date, f)
    ed = dt.datetime.strptime(end_date, f)
    dates = arrange_dates(sd, ed)
    candidates = ['Biden', 'Trump']
    
    # fetch tweets & save CSVs (each CSV contains one day/candidate)
    
    lam_plus_1 = lambda x: (x + dt.timedelta(days=1)).strftime("%Y-%m-%d")
    wait_time = 60 # seconds; will use for sleep timer to avoid 'client payload' error
    
    for date in dates:
        for candidate in candidates:
            
            d = date.strftime("%Y-%m-%d")            
            print(f"""\n Begin scraping {candidate}'s tweets on {d}\n \n""")
            outfile = f"./pickle/tweets/{candidate}_{d}.csv"
            
            # twint search query details
            c = twint.Config()
            c.Search = candidate
            c.Lang = 'en'
            c.Since = d
            c.Until = lam_plus_1(date)
            c.Limit = tweets_per_day
            c.Store_csv = True
            c.Output = outfile
            
            # run the search
            twint.run.Search(c)
            print(f"{outfile} created.")
            
            # wait some time so script does not receive 'client payload' error
            print(f"Waiting {wait_time} seconds...")
            time.sleep(wait_time) # to avoid 'client payload' error
            wait_time += 10 * random.uniform(0, 1)

In [ ]:
get_election_tweets(start_date="2020-10-12", end_date="2020-11-03")


 Begin scraping Biden's tweets on 2020-10-12]n 

1315804441291808769 2020-10-12 23:59:59 +0000 <ItsAdele19> @LepapillonBlu @the_first_party You know what’s great about Biden/Harris supporters?  Biden/Harris supporters are hardworking. They do the work. Passionately.   Unlike Trump supporters who just want to loiter about at rallies, play with their guns, and stand around waiting to yell at someone for wearing a mask.
1315804440910336005 2020-10-12 23:59:59 +0000 <petermi51593775> @TeamTrump @realDonaldTrump Groundhog thinks this is going to upset Barack Obama cuz he wants isis to real strong and kill American soldiers and Americans and Christians all over the world and burn their churches down Republicans keep America safe! Joe Biden keeps terrorism strong!  https://t.co/oMeJR37dOk
1315804440755089408 2020-10-12 23:59:59 +0000 <RickB57198384> Biden is delusional. The Dems are racist and their party has a history of racism they didnt abolish the slaves the Republicans did. The Dems fou

1315804384211668992 2020-10-12 23:59:46 +0000 <Gamarays4> @h_vyper @JoeBiden No that’s what you liberals do.
1315804384039710721 2020-10-12 23:59:46 +0000 <Kamunt> McConnell reaffirms McGrath will cede Senate power to Schumer/NY, says she, like Biden, want to pack SCOTUS, which is proven by their inability to address the issue.  They both get one last 30-sec response, but they basically just restate their answers.  Debate over. #KYSen
1315804383179755521 2020-10-12 23:59:46 +0000 <websitenut> @LisaHagan7 Yes. Which means trump knew he had it much earlier and deliberately tried to expose Joe Biden at the debate. That’s why he talked incessantly and loudly at Joe. He openly tried to murder him!
1315804382903050240 2020-10-12 23:59:46 +0000 <GoCCal54> @realDonaldTrump VOTE BIDEN
1315804381246377985 2020-10-12 23:59:45 +0000 <jilstereo> Sure.  Ok.  Believe.  Biden said again he’s running for Senate.
1315804380407324672 2020-10-12 23:59:45 +0000 <PDXFato> @MatthewJshow @no_silenced @JoeBide

1315804329241194504 2020-10-12 23:59:33 +0000 <DRAINSWAMP2024> @KamalaHarris @BarackObama @JoeBiden At least I know where my skyrocketing premiums are going these past years.
1315804328968548352 2020-10-12 23:59:33 +0000 <spunkkee> 👍On 10/10/20 I voted for a straight Biden ticket &amp; NO MORE GOP! Used NJ Dropbox! We must seat  competent HONEST LEADERS! Vote in person with paper trail OR hand postman ballot OR use registrar ballot dropbox. Indict impeached trump on RICO, tax evasion, treason, rape, &amp; fraud.
1315804328247144450 2020-10-12 23:59:32 +0000 <ChicagoPhotoSho> Your racist uncle and all those fb qanon meme people do not participate in polls. But they will vote. Do not think Biden has this wrapped up. You have to vote.
1315804328041549826 2020-10-12 23:59:32 +0000 <wondershowers05> @threadreaderapp @8R4D7EY @mpukita @JoeBiden @SaraCarterDC @paulsperry_ @realDonaldTrump @GretaMiddendor1 @WitmerCarl @fedagentmark @MsfFlynnerland @helpmerhonda49 @mbolch @helene713 @RebelSiste

1315804257086365697 2020-10-12 23:59:16 +0000 <Teetsjones> @JoeBiden The longer you aren't the better offthe US is.
1315804256801259525 2020-10-12 23:59:15 +0000 <AlextheGreatGay> @mellystardust Yes, I thought that educating and mobilizing leftist action was also the goal. I'm not sure if I'm following the right people though because I mostly just see people talking about how terrible Biden is and whether they'll vote Green. I kinda thought there would be more than that.
1315804255966638081 2020-10-12 23:59:15 +0000 <mckeag_brent> @JoeBiden You can't even remember what you're running for. This is the 2nd time you stated you're running for the Senate. Just stop.
1315804255421427712 2020-10-12 23:59:15 +0000 <Jillian66863250> @JoeBiden Everyday for 47 years .lol VOTE TRUMP2020
1315804253890514954 2020-10-12 23:59:15 +0000 <armysman30> @charliekirk11 I see build back better, and think is just another way to say MAGA, but in a way that will for sure be screwed up by Biden if he try’s to re

1315804196348801024 2020-10-12 23:59:01 +0000 <182Fred> @winkle_ms @nickgreatpwrful @BrandonStraka @JoeBiden I did not say anything about anything to do with Christianity. I'm guessing that was your persecution complex kicking in.   And of course it was a moral issue. There was incredibly little factual input to support the decision and the DoD was blindsided and confused as to why.
1315804195841363970 2020-10-12 23:59:01 +0000 <mario_mora11> @paas2308 @desbordes Candidato Biden,esta compitiendo para senador,el mismo lo dijo en un video rreciente,Ya nada sorprende
1315804195795079168 2020-10-12 23:59:01 +0000 <DeadManOnAStick> @BobRogers10 @kathfyt @Brick_Suit @JoeBiden No, you very clearly don't. Either that or you don't actually understand all the actions they've taken.
1315804195262472192 2020-10-12 23:59:01 +0000 <sunshinetrader> @HKrassenstein @realDonaldTrump Biden is 100% losing Florida.
1315804194591272960 2020-10-12 23:59:01 +0000 <Dollop_of_Mayo> @Agent47Log @FunRageDie @Kama

...

1315801463105351680 2020-10-12 23:48:09 +0000 <TroublesomeF> @DucksForDuckGod @RTodKelly It's like he and Trump were one generic dude and then they split like Danny DeVito and Arnold Schwarzenegger in Twins, with Biden getting the likeable parts of the personality.
1315801462992252930 2020-10-12 23:48:09 +0000 <lesmagrittes> @samanthamarika1 The other Biden.
1315801461624721408 2020-10-12 23:48:09 +0000 <MOStarWarsStuff> @realDonaldTrump Vote Biden!
1315801460790095872 2020-10-12 23:48:09 +0000 <Tikka300mag> @KamalaHarris @BarackObama @JoeBiden Yea, they penalized those who didn't have it. Peoples deductibles more than tripled, premiums doubled and most plans coverage went from 90/10 and 80/20 to 50/50.  Sounds like to me they were lining their pockets as well as Insurance companies.
1315801460286918661 2020-10-12 23:48:09 +0000 <Thereal_ssteele> @RealJamesWoods ...you forgot to add so Biden can spend a quiet evening in with his sister.
1315801459427078144 2020-10-12 23:48:08 +0000 <js


 Begin scraping Trump's tweets on 2020-10-12]n 

1315804441291808769 2020-10-12 23:59:59 +0000 <ItsAdele19> @LepapillonBlu @the_first_party You know what’s great about Biden/Harris supporters?  Biden/Harris supporters are hardworking. They do the work. Passionately.   Unlike Trump supporters who just want to loiter about at rallies, play with their guns, and stand around waiting to yell at someone for wearing a mask.
1315804439983337472 2020-10-12 23:59:59 +0000 <holadayNexpress> @rickklein Is Trump running for Senate ? Same as Biden
1315804439689736198 2020-10-12 23:59:59 +0000 <gartmartin9> @JakeSherman what city is Trump from?
1315804439635255296 2020-10-12 23:59:59 +0000 <yongz_> @ddale8 Do Trump supporters care about his lies or are they too stupid?
1315804439450710020 2020-10-12 23:59:59 +0000 <punchie41> Trump at rallies. More infections=dead Americans. 214000 deaths, &amp; they are not even enforcing safety. #VoteOutTrump OR DIE. @AARP @AAAnews @dnc @gop @cnn @msnbc @ap @abc @

1315804405967597571 2020-10-12 23:59:51 +0000 <FR24NewsFrance> Trump sous tension lors du premier grand rassemblement électoral depuis la reprise du COVID-19  https://t.co/XI8eXFyT2e
1315804405904670721 2020-10-12 23:59:51 +0000 <AviellaLloyd> Utah high school student bullied, spat on for wearing hat in support of President Trump  https://t.co/LPHbEdNBg0
1315804405082451969 2020-10-12 23:59:51 +0000 <RowanMarxFilms> Yeah sure, Trump didn't do anything about Russia putting bounties on US soldiers. The soldiers shouldn't be there anyway lol.
1315804404583464960 2020-10-12 23:59:51 +0000 <spice_is_nicee> @daviru @GooseGanderMom @nytimes Yes... tax filings especially for corporations are extremely complicated and the IRS itself said they don't have the resources to audit complex filings so they simply don't. trump could release his tax returns like be promised but he won't. Guess why   https://t.co/zX9KBV7AVZ
1315804404512030721 2020-10-12 23:59:51 +0000 <StandForBetter> What's Trump doing

1315804373113434112 2020-10-12 23:59:43 +0000 <chilibowlcheryl> @jfreewright The Yo-Yos: Jr. &amp; Eric on uppers.  Ivanka &amp; Melania speak as if they are on downers.  Trump-uppers when riffing, downers when reading.
1315804372559892486 2020-10-12 23:59:43 +0000 <jeff340hpd> @ProjectLincoln Well we will see when he doesn’t get it again but you’re a liar.  The scientists and doctors on CNN have stated you get some form of immunity through T cells at least. But that was before Trump got it. Hilarious
1315804372392120321 2020-10-12 23:59:43 +0000 <KarensComebacks> @yashar After watching Trump play golf for 1/4 of his term I couldn't care less what Cuomo is up to.  He was right there when the crisis was peaking and daily kept working on it.  Trump did nothing and continues to do nothing.
1315804372304113665 2020-10-12 23:59:43 +0000 <mike_paroulek> @ddale8 For Trump so Loved the World, He Spread His Powerful Herd Mentality Herd Immunity Upon the Masses. Praise Him and Bask in his Presen

1315804334563753984 2020-10-12 23:59:34 +0000 <ClaimsStop> Objective voters know facts from fiction! Getting rid of #Trump is getting rid of #Trumpism, a fascist cult that is the antithesis of conservatism!
1315804334375006208 2020-10-12 23:59:34 +0000 <Texasstarrr> @HoustonChron Outside of his donor class, John Cornyn has done nothing for the average Texan.  He is Trump’s puppet and has no moral courage to stand up to him.
1315804334303727616 2020-10-12 23:59:34 +0000 <shanemontoya42> @csilenz @psychoticangel I read the first three claims being fact checked and they actually included the Russian hoax. Please dont send links of the people who hate Trump and lie about him.
1315804334114762752 2020-10-12 23:59:34 +0000 <EmbacubaNZ> Real Clear Politics National Average: Biden 51,8, Trump 41,6. Attention. A new war may be coming.#Cuba  https://t.co/mV1CC3OZrv
1315804334010044416 2020-10-12 23:59:34 +0000 <jp_bolo4jp> @eugenegu @realDonaldTrump Wow, you agreed with Trump! You are finally sh